# Diffuse

> Transforms an image based on a given prompt.  

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp diffuse

In [3]:
#| export
import numpy as np
from PIL import Image
import cv2
from fastcore.basics import store_attr
from diffusers import AutoPipelineForImage2Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
torch.set_default_device("mps")
from diffusers.utils import load_image
from controlnet_aux import OpenposeDetector
from mmm_st.config import Config
from PIL import Image
from diffusers import KandinskyV22PriorEmb2EmbPipeline, KandinskyV22ControlnetImg2ImgPipeline
from transformers import pipeline


In [4]:
#| export
class BaseTransformer:
    def __init__(
            self,
            model_name=Config.MODEL_NAME,
            device=Config.DEVICE,
            img_size=Config.IMG_SIZE,
            num_steps=Config.NUM_STEPS,
        ):
        store_attr()
        self._initialize_pipeline()

    def _initialize_pipeline(self):
        raise NotImplementedError("Subclasses should implement this method.")

    def transform_image(self, image, prompt):
        raise NotImplementedError("This method should be overridden by subclasses.")

    def prepare_image(self, image):
        """ Prepare the image for transformation, converting it to the correct size and format. """
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        return image.resize(self.img_size)


In [ ]:
#| export
class ImageTransformer(BaseTransformer):
    """
    A class to manage image transformation using the Hugging Face diffusers pipeline.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        store_attr()  # Store initialization parameters for easy access later
        self._initialize_pipeline()

    def _initialize_pipeline(self):
        """
        Initializes the img2img pipeline with model settings optimized for performance.
        """
        pipe = AutoPipelineForImage2Image.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to(self.device)
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
        pipe.enable_model_cpu_offload()
        self.pipeline = pipe
        

    def transform_image(self, image, prompt):
        """
        Transforms an image based on a given prompt using a pre-trained model.

        Args:
            image: A numpy array of the image to transform.
            prompt (str): The creative prompt to guide the image transformation.

        Returns:
            PIL.Image: The transformed image.
        """
        image = self.prepare_image(image)

        return self.pipeline(
            prompt=prompt, 
            image=image,
        ).images[0]

In [5]:
#| export
class EdgeImageTransformer(BaseTransformer):
    def __init__(
            self,
            *args, 
            canny_control_model,
            low_threshold=100,
            high_threshold=200,
            **kwargs,
        ):
        store_attr()
        super().__init__(*args, **kwargs)

    def _initialize_pipeline(self):
        controlnet = ControlNetModel.from_pretrained(self.canny_control_model, torch_dtype=torch.float16)
        pipe = StableDiffusionControlNetPipeline.from_pretrained(
            self.model_name,
            controlnet=controlnet,
            torch_dtype=torch.float16
        ).to(self.device)
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
        pipe.enable_model_cpu_offload()
        self.pipeline = pipe

    def transform_image(self, image, prompt):
        edge_image = cv2.Canny(image, self.low_threshold, self.high_threshold)
        edge_image = np.stack((edge_image, edge_image, edge_image), axis=-1)
        edge_image = self.prepare_image(edge_image)
        return self.pipeline(
            prompt=prompt,
            images=[edge_image],
            num_inference_steps=self.num_steps,
        ).images[0]

In [7]:
#| export
class PoseImageTransformer(BaseTransformer):
    def __init__(
            self,
            *args,
            pose_control_model=Config.CONTROL_NET_CANNY_MODEL,
            pose_det_model=Config.POSE_DET_MODEL,
            **kwargs,
        ):
        store_attr()
        super().__init__(*args, **kwargs)

    def _initialize_pipeline(self):
        self.pose_model = OpenposeDetector.from_pretrained(self.pose_det_model)
        controlnet = ControlNetModel.from_pretrained(self.pose_control_model, torch_dtype=torch.float16)
        pipe = StableDiffusionControlNetPipeline.from_pretrained(
            self.model_name,
            controlnet=controlnet,
            torch_dtype=torch.float16
        ).to(self.device)
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
        pipe.enable_model_cpu_offload()
        self.pipeline = pipe

    def transform_image(self, image, prompt):
        pose_image = self.pose_model(image)
        pose_image = self.prepare_image(pose_image)
        return self.pipeline(
            prompt=prompt, 
            images=[pose_image], 
            num_inference_steps=self.num_steps,
        ).images[0]


In [8]:
#| export
class CombinedImageTransformer(BaseTransformer):
    def __init__(
            self,
            *args, 
            canny_control_model,
            pose_control_model,
            pose_det_model,
            low_threshold=100,
            high_threshold=200,
            **kwargs,
        ):
        store_attr()
        super().__init__(*args, **kwargs)

    def _initialize_pipeline(self):
        self.pose_model = OpenposeDetector.from_pretrained(self.pose_det_model)
        controlnet = [
            ControlNetModel.from_pretrained(self.pose_control_model, torch_dtype=torch.float16),
            ControlNetModel.from_pretrained(self.canny_control_model, torch_dtype=torch.float16),
        ]
        pipe = StableDiffusionControlNetPipeline.from_pretrained(
            self.model_name,
            controlnet=controlnet,
            torch_dtype=torch.float16
        ).to(self.device)
        pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
        pipe.enable_model_cpu_offload()
        self.pipeline = pipe

    def transform_image(self, image, prompt):
        prepared_pose_image = self.prepare_image(self.pose_model(image))
        edge_image = cv2.Canny(image, self.low_threshold, self.high_threshold)
        edge_image = np.stack((edge_image, edge_image, edge_image), axis=-1)
        prepared_canny_image = self.prepare_image(edge_image)
        return self.pipeline(
            prompt=prompt,
            images=[prepared_pose_image, prepared_canny_image],
            negative_prompt=[Config.NEGATIVE_PROMPT],
            num_inference_steps=self.num_steps,
            controlnet_conditioning_scale=[1.0, 0.8]
        ).images[0]

In [ ]:
#| export

class KandinskyTransformer(BaseTransformer):
    def __init__(
            self,
            *args, 
            model_name=Config.MODEL_NAME,
            device=Config.DEVICE,
            img_size=Config.IMG_SIZE,
            num_steps=Config.NUM_STEPS,
            **kwargs,
        ):
        store_attr()
        super().__init__(*args, model_name=model_name, device=device, img_size=img_size, num_steps=num_steps, **kwargs)
        self._initialize_pipeline()

    def _initialize_pipeline(self):
        self.depth_estimator = pipeline("depth-estimation", device=self.device)
        self.prior_pipeline = KandinskyV22PriorEmb2EmbPipeline.from_pretrained(
            "kandinsky-community/kandinsky-2-2-prior",
            torch_dtype=torch.float16,
            use_safetensors=True,
        ).to(self.device)

        self.pipeline = KandinskyV22ControlnetImg2ImgPipeline.from_pretrained(
            "kandinsky-community/kandinsky-2-2-controlnet-depth",
            torch_dtype=torch.float16,
            use_safetensors=True,
        ).to(self.device)
        self.pipeline.enable_model_cpu_offload()

    def make_hint(self, image):
        image = self.depth_estimator(image)["depth"]
        image = np.array(image)
        image = image[:, :, None]
        image = np.concatenate([image, image, image], axis=-1)
        detected_map = torch.from_numpy(image).float() / 255.0
        hint = detected_map.permute(2, 0, 1)
        return hint

    def transform_image(self, image, prompt, negative_prompt, generator):
        img = self.prepare_image(image)
        hint = self.make_hint(img).unsqueeze(0).half().to(self.device)
        
        img_emb = self.prior_pipeline(prompt=prompt, image=img, strength=0.85, generator=generator)
        negative_emb = self.prior_pipeline(prompt=negative_prompt, image=img, strength=1, generator=generator)
        
        result_image = self.pipeline(
            image=img, 
            strength=0.7, 
            image_embeds=img_emb.image_embeds, 
            negative_image_embeds=negative_emb.image_embeds, 
            hint=hint, 
            num_inference_steps=self.num_steps, 
            generator=generator, 
            height=self.img_size[0], 
            width=self.img_size[1],
        ).images[0]
        return result_image

In [9]:
#|export
def get_transformer(transformer_type):
    if transformer_type == "regular":
        return ImageTransformer()
    elif transformer_type == "canny":
        return EdgeImageTransformer()
    elif transformer_type == "pose":
        return PoseImageTransformer()
    elif transformer_type == "combined":
        return CombinedImageTransformer()
    else:
        raise ValueError("Unknown transformer type provided")


In [4]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers import UNet2DConditionModel, AutoencoderKL
from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker
from transformers import AutoTokenizer, AutoFeatureExtractor, CLIPTextModel, CLIPVisionModel
from diffusers.utils import load_image
import numpy as np
from PIL import Image
import cv2


In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()